# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing Turi Create functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in Turi Create functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('./data/home_data.sframe/')

In [3]:
sales.head(1)

Columns:
	id	str
	date	datetime
	price	float
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	float
	floors	float
	waterfront	int
	view	int
	condition	int
	grade	float
	sqft_above	float
	sqft_basement	float
	yr_built	float
	yr_renovated	float
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float

Rows: 1

Data:
+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade |
+-------------+----------+--------+------------+------+-----------+-------+
|    1180.0   |  5650.0  |  1.0   |     0      |  0   |     3     |  7.0  |
+-------------+----------+--------+------------+------+-----------+-------+
+------------+---------------+----------+--------------+---------+-------------+
| sqft_above | sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+------------+---------------+----------+--------------+---------+-------------+
|   1180.0   |      0.0      |  1955.0  |     0.0      |  98178  | 47.51123398 |
+------------+---------------+----------+--------------+---------+-------------+
+---------------+---------------+-----+
|      long     | sqft_living15 | ... |
+---------------+---------------+-----+
| -122.25677536 |     1340.0    | ... |
+---------------+---------------+-----+
[1 rows x 21 columns]

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let Turi Create pick a random seed for you).  

In [4]:
train_data,test_data = sales.random_split(.8, seed=0)

In [5]:
train_data.shape, test_data.shape

((17384, 21), (4229, 21))

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [6]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = turicreate.linear_regression.create(train_data, 
                                                    target='price', 
                                                    features=example_features, 
                                                    validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training Max Error | Training Root-Mean-Square Error |

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 1.028694     | 4146407.600631     | 258679.804477                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [7]:
example_weight_summary = example_model.coefficients
example_weight_summary

name,index,value,stderr
(intercept),None,87910.07249239634,7873.338143401682
sqft_living,None,315.40344055209886,3.4557003258547367
bedrooms,None,-65080.21555282676,2717.456854420713
bathrooms,None,6944.020192638501,3923.114931441493


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing Turi Create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [8]:
example_predictions = example_model.predict(train_data)
example_predictions[0]  # should be 271789.505878

271789.5058780312

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [9]:
def get_residual_sum_of_squares(model, data, outcome):
    preds = model.predict(data)   # First get the predictions
    diff = outcome - preds        # Then compute the residuals/errors
    rss = (diff * diff).sum()     # Then square and add them up
    return rss

Test your function by computing the RSS on TEST data for the example model:

In [10]:
Eps = 1e3

rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
exp_value = 2.7376153833e+14  # should be 2.7376153833e+14
assert abs(rss_example_train - exp_value) <= Eps 

print(f"rss_example_train is: {rss_example_train:2.7e}") # should be 2.7376153833e+14

rss_example_train is: 2.7376154e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [11]:
from math import log

Next create the following 4 new features as column in both *test* and *train* data:
* bedrooms_squared = bedrooms \* bedrooms
* bed_bath_rooms = bedrooms \* bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [12]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [13]:
# create the remaining 3 features in both TEST and TRAIN data
for tdata in (train_data, test_data):
  for (f1, f2, lab, fn) in [
    ('bedrooms', 'bathrooms', 'bed_bath_rooms', lambda r: r['bedrooms'] * r['bathrooms']), # lambda t: t[0] * t[1]), 
    ('sqft_living', None, 'log_sqft_living', lambda x: log(x)), 
    ('lat', 'long', 'lat_plus_long', lambda r: r['lat'] + r['long'])]: # lambda t: t[0] + t[1])]:
  
    if f2 is None:
      tdata[lab] = tdata[f1].apply(fn)
    else:
      tdata[lab] = tdata.apply(fn)

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

In [14]:
train_data.head(2)

Columns:
	id	str
	date	datetime
	price	float
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	float
	floors	float
	waterfront	int
	view	int
	condition	int
	grade	float
	sqft_above	float
	sqft_basement	float
	yr_built	float
	yr_renovated	float
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float
	bedrooms_squared	float
	bed_bath_rooms	float
	log_sqft_living	float
	lat_plus_long	float

Rows: 2

Data:
+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 | 221900.0 |   3.0    |    1.0    |
| 6414100192 | 2014-12-09 00:00:00+00:00 | 538000.0 |   3.0    |    2.25   |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade |
+-------------+----------+--------+------------+------+-----------+-------+
|    1180.0   |  5650.0  |  1.0   |     0      |  0   |     3     |  7.0  |
|    2570.0   |  7242.0  |  2.0   |     0      |  0   |     3     |  7.0  |
+-------------+----------+--------+------------+------+-----------+-------+
+------------+---------------+----------+--------------+---------+-------------+
| sqft_above | sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+------------+---------------+----------+--------------+---------+-------------+
|   1180.0   |      0.0      |  1955.0  |     0.0      |  98178  | 47.51123398 |
|   2170.0   |     400.0     |  1951.0  |    1991.0    |  98125  | 47.72102274 |
+------------+---------------+----------+--------------+---------+-------------+
+---------------+---------------+-----+
|      long     | sqft_living15 | ... |
+---------------+---------------+-----+
| -122.25677536 |     1340.0    | ... |
|  -122.3188624 |     1690.0    | ... |
+---------------+---------------+-----+
[2 rows x 25 columns]

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on *test* data? (round to 2 digits)**

In [15]:
for attr in ['bed_bath_rooms', 'log_sqft_living', 'lat_plus_long', 'bedrooms_squared']:
  print(f"mean of {attr:16s} is: {test_data[attr].mean():3.2f}")

mean of bed_bath_rooms   is: 7.50
mean of log_sqft_living  is: 7.55
mean of lat_plus_long    is: -74.65
mean of bedrooms_squared is: 12.45


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [16]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [17]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = turicreate.linear_regression.create(train_data, 
                                              target='price', 
                                              features=model_1_features, 
                                              validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.041668     | 4074878.213083     | 236378.596455                   |

| 2         | 3        | 0.082464     | 4074878.213108     | 236378.596455                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [18]:
model_2 = turicreate.linear_regression.create(train_data, 
                                              target='price', 
                                              features=model_2_features, 
                                              validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 6

Number of unpacked features : 6

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.049402     | 4014170.932931     | 235190.935428                   |

| 2         | 3        | 0.088721     | 4014170.932936     | 235190.935429                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [19]:
model_3 = turicreate.linear_regression.create(train_data, 
                                              target='price', 
                                              features=model_3_features, 
                                              validation_set=None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 9

Number of unpacked features : 9

...

+-----------+----------+--------------+--------------------+---------------------------------+

| 1         | 2        | 0.015378     | 3193229.177893     | 228200.043155                   |

| 2         | 3        | 0.030031     | 3193229.177873     | 228200.043155                   |

+-----------+----------+--------------+--------------------+---------------------------------+

SUCCESS: Optimal solution found.

In [20]:
# Examine/extract each model's coefficients:
model_1.coefficients

name,index,value,stderr
(intercept),None,-56140675.741143875,1649985.420756234
sqft_living,None,310.2633257769216,3.1888296040912327
bedrooms,None,-59577.116067596784,2487.2797732247486
bathrooms,None,13811.840541652928,3593.5421329742026
lat,None,629865.7894714862,13120.710032179959
long,None,-214790.28516470612,13284.285165198724


In [21]:
model_2.coefficients

name,index,value,stderr
(intercept),None,-54410676.111842,1650405.1658980648
sqft_living,None,304.44929805557956,3.202175356388602
bedrooms,None,-116366.04322944915,4805.549665490957
bathrooms,None,-77972.33051298966,7565.05991098361
lat,None,625433.8349398081,13058.353097099072
long,None,-203958.6029374751,13268.12837566144
bed_bath_rooms,None,26961.62490895251,1956.3656155643494


In [22]:
model_3.coefficients

name,index,value,stderr
(intercept),None,-52974974.06892096,1615194.9438646361
sqft_living,None,529.1964205687524,7.699134985086644
bedrooms,None,28948.527746351323,9395.728891134848
bathrooms,None,65661.2072396969,10795.338070248838
lat,None,704762.1484430908,690691861.1792433
long,None,-137780.02000716343,690691861.177811
bed_bath_rooms,None,-8478.36410716755,2858.9539125648853
bedrooms_squared,None,-6072.384661905014,1494.9704277846301
log_sqft_living,None,-563467.7842801772,17567.823081294748
lat_plus_long,None,-83217.197910035,690691861.1039315


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**
  - [x] Positive (+)
  - [ ] Negative (-)


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**
  - [ ] Positive (+)
  - [x] Negative (-)
  
Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on *training* Data for each of the three models.

In [23]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
def calc_rss(models=[('model_1', model_1), ('model_2', model_2), ('model_3', model_3)], 
                  tdata=train_data):
  for (mname, mod) in models:
    rss_mod = get_residual_sum_of_squares(mod, tdata, tdata['price'])
    print(f"model {mname:10s} rss is: {rss_mod:2.7e}")
  return

calc_rss() # fro all models on train set 

model model_1    rss is: 9.7132823e+14
model model_2    rss is: 9.6159207e+14
model model_3    rss is: 9.0527631e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

  - [ ] Model 1
  - [ ] Model 2
  - [x] Model 3
  
  
Yes I would expect this (on training data)

Now compute the RSS on on *test* data for each of the three models.

In [24]:
# Compute the RSS on TESTING data for each of the three models and record the values:
calc_rss(tdata=test_data) 

model model_1    rss is: 2.2656809e+14
model model_2    rss is: 2.2436880e+14
model model_3    rss is: 2.5182932e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

  - [ ] Model 1
  - [x] Model 2
  - [ ] Model 3